In [74]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split

In [75]:
import os

def load_data():
    component = pd.read_csv("../dataset/complete_dataset.csv")
    #component = component.drop(['Project','Version','Smell'],axis=1)
    df = pd.DataFrame()

    path = "../../projects"
    projects = []

    # Itera su tutti i file e le cartelle nella cartella specificata
    for item in os.listdir(path):
        # Se l'elemento nella cartella è una cartella, aggiungi il nome alla lista
        if os.path.isdir(os.path.join(path, item)):
            projects.append(item)
    # Stampa la lista di nomi dei progetti
    print(projects)

    possible_subfolders = ["src/java", "src/main", "src/main/java", "src"]
    #classes = []

    for k in range(len(projects)):
        print(projects[k])
        for i in range(len(component)):
            line = component.loc[i,'ComponentName'].strip()
            line = line.replace(".","/") + ".java"
            for subfolder in possible_subfolders:
                full_path = os.path.join(path, projects[k], subfolder, line)
                if os.path.exists(full_path):
                    with open(full_path, "r") as f:
                        contenuto = f.read()
                    # classes.append(contenuto)
                    temp_df = pd.DataFrame({
                        'Project_name': projects[k], 
                        'Component': [contenuto],
                        'CDSBP' : [component.loc[i, 'CDSBP']],
                        'CC': [component.loc[i,'CC']],
                        'LC': [component.loc[i,'LC']],
                        'LZC': [component.loc[i,'LZC']],
                        'RB': [component.loc[i,'RB']],
                        'SC': [component.loc[i,'SC']]
                    })
                    df = pd.concat([df, temp_df], ignore_index=True)
                    break # Esci dal ciclo for se hai trovato il file
    return df

In [76]:
path = "../../projects"
projects = []
code_smells = ['CDSBP', 'CC', 'LC', 'LC', 'LZC', 'RB', 'SC']
# Itera su tutti i file e le cartelle nella cartella specificata
for item in os.listdir(path):
    # Se l'elemento nella cartella è una cartella, aggiungi il nome alla lista
    if os.path.isdir(os.path.join(path, item)):
       projects.append(item)
    # Stampa la lista di nomi dei progetti
print(projects)

['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0', 'lucene-releases-lucene-solr-3.6.0', 'manifold-cf-release-0.6', 'nutch-release-1.4', 'pig-release-0.8.0', 'qpid-0.14', 'struts-STRUTS_2_3_4', 'xerces2-j-Xerces-J_2_3_0']


In [77]:
df = load_data()

['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0', 'lucene-releases-lucene-solr-3.6.0', 'manifold-cf-release-0.6', 'nutch-release-1.4', 'pig-release-0.8.0', 'qpid-0.14', 'struts-STRUTS_2_3_4', 'xerces2-j-Xerces-J_2_3_0']
ant-ivy-2.0.0-alpha2
ant-rel-1.8.3
cassandra-cassandra-1.0.0
elasticsearch-v0.19.0
hadoop-release-0.6.0
hive-release-0.9.0
hsqldb-2.2.8
karaf-karaf-2.3.0
lucene-releases-lucene-solr-3.6.0
manifold-cf-release-0.6
nutch-release-1.4
pig-release-0.8.0
qpid-0.14
struts-STRUTS_2_3_4
xerces2-j-Xerces-J_2_3_0


In [78]:
df

,Project_name,Component,CDSBP,CC,LC,LZC,RB,SC
0,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
1,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
2,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
3,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
4,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
8139,xerces2-j-Xerces-J_2_3_0,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,0
8140,xerces2-j-Xerces-J_2_3_0,// XMLFilterImpl.java - base SAX2 filter imple...,0,0,0,0,0,0
8141,xerces2-j-Xerces-J_2_3_0,// XMLReaderAdapter.java - adapt an SAX2 XMLRe...,0,0,0,0,0,0
8142,xerces2-j-Xerces-J_2_3_0,// XMLReaderFactory.java - factory for creatin...,0,0,0,0,0,1


In [79]:
# Dizionario per salvare le predizioni
predictions = {}

# Dizionario per salvare le metriche
metrics = {}

# Loop attraverso i progetti
for project in projects:
    print("Current project:", project)

    # Caricamento dei dati specifici del progetto
    project_df = df[df['Project_name'] == project]

    # Inizializza il dizionario per il progetto corrente
    predictions[project] = {}
    metrics[project] = {}

    # Loop attraverso i code smell
    for smell in code_smells:
        print("Current code smell:", smell)

        X = project_df["Component"]
        y = project_df[str(smell)]

        if len(project_df) == 0:
            continue

        # Effettua lo split in train e test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

        # Definisci il classificatore
        classifier = MLPClassifier(verbose=False, hidden_layer_sizes=[10, 20, 20])

        # Crea il pipeline
        pipeline = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('classifier', classifier),
        ], verbose=False)

        # Addestra il modello
        pipeline.fit(X_train, y_train)

        # Valuta il modello
        y_pred = pipeline.predict(X_test)

        # Salva le predizioni per lo smell corrente nel progetto corrente
        predictions[project][smell] = y_pred

        precision = precision_score(y_test, y_pred, average="macro")
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average="macro")

        # Salva le metriche per lo smell corrente nel progetto corrente
        metrics[project][smell] = {
            "Precision": precision,
            "Accuracy": accuracy,
            "Recall": recall
        }

        print("Accuracy:", accuracy)
        print("Recall:", recall)
        print("Precision:", precision)
        print("--------------")

Current project: ant-ivy-2.0.0-alpha2
Current code smell: CDSBP


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: CC


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: LC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: LC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: LZC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: RB
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: SC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current project: ant-rel-1.8.3
Current code smell: CDSBP
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: CC


...

Accuracy: 0.9961389961389961
Recall: 0.5
Precision: 0.4980694980694981
--------------
Current code smell: LZC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current code smell: RB
Accuracy: 0.9806949806949807
Recall: 0.7123015873015873
Precision: 0.8671568627450981
--------------
Current code smell: SC
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
--------------
Current project: qpid-0.14
Current code smell: CDSBP
Current code smell: CC
Current code smell: LC
Current code smell: LC
Current code smell: LZC
Current code smell: RB
Current code smell: SC
Current project: struts-STRUTS_2_3_4
Current code smell: CDSBP
Current code smell: CC
Current code smell: LC
Current code smell: LC
Current code smell: LZC
Current code smell: RB
Current code smell: SC
Current project: xerces2-j-Xerces-J_2_3_0
Current code smell: CDSBP
Accuracy: 0.9942196531791907
Recall: 0.9848484848484849
Precision: 0.9964539007092199
--------------
Current code smell: CC


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.976878612716763
Recall: 0.8718944099378882
Precision: 0.94079754601227
--------------
Current code smell: LC
Accuracy: 0.9826589595375722
Recall: 0.6860207100591715
Precision: 0.8676900584795322
--------------
Current code smell: LC
Accuracy: 0.9826589595375722
Recall: 0.8690828402366864
Precision: 0.7970238095238096
--------------
Current code smell: LZC


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9913294797687862
Recall: 0.5
Precision: 0.4956647398843931
--------------
Current code smell: RB
Accuracy: 0.9826589595375722
Recall: 0.9239560439560439
Precision: 0.9239560439560439
--------------
Current code smell: SC
Accuracy: 0.9942196531791907
Recall: 0.96003696003696
Precision: 0.96003696003696
--------------


In [80]:
# Stampa le predizioni per ogni progetto e smell
for project, smell_predictions in predictions.items():
    print("Project:", project)
    for smell, pred_list in smell_predictions.items():
        print("Code Smell:", smell)
        print(pred_list)
        print("--------------")


Project: ant-ivy-2.0.0-alpha2
Code Smell: CDSBP
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: CC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: LC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: LZC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
# Stampa le metriche per ogni progetto e smell
for project, smell_metrics in metrics.items():
    print("Project:", project)
    for smell, metric_values in smell_metrics.items():
        print("Code Smell:", smell)
        print(metric_values)
        print("--------------")

#### Il CountVectorizer genera una rappresentazione vettoriale in cui ogni parola unica diventa una feature e la frequenza di ciascuna parola viene conteggiata. Il TfidfTransformer, invece, calcola la frequenza termine-frequenza inversa del documento (TF-IDF) delle feature estratte dal CountVectorizer. Il TF-IDF è una misura statistica che tiene conto sia della frequenza della parola in un documento (TF) che della sua rarità nell'intero corpus (IDF). Questo aiuta a pesare le parole più significative rispetto a quelle comuni. Il TfidfTransformer moltiplica la matrice delle frequenze delle parole per il peso TF-IDF per ottenere una rappresentazione vettoriale ponderata.